In [1]:
from robot_library_py import *
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Joy, JointState
from intera_core_msgs.msg import JointCommand

from threading import Thread
import time
import numpy as np 
from std_msgs.msg import Bool
import rotm2euler
 
import cv2
from matplotlib import pyplot as plt
import pickle
import datetime
import os 
import argparse

import argparse
import json
import numpy as np
import time
import os
import shutil
import psutil
import sys
import socket
import traceback

from collections import OrderedDict

import torch
from torch.utils.data import DataLoader

import robomimic
import robomimic.utils.train_utils as TrainUtils
import robomimic.utils.torch_utils as TorchUtils
import robomimic.utils.obs_utils as ObsUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.file_utils as FileUtils
from robomimic.config import config_factory
from robomimic.algo import algo_factory, RolloutPolicy
from robomimic.utils.log_utils import PrintLogger, DataLogger, flush_warnings

import matplotlib.pyplot as plt
import h5py

import os
import time 

from utils_ros import ROSInterface
import traceback

ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /home/carl/robomimic/robomimic/scripts/setup_macros.py
)


In [2]:
import beepy as beep
beep_start = lambda : beep.beep('coin')

In [3]:
tb=Thread(target=beep_start)
tb.start()

In [4]:
ckpt_path = "/home/carl/data_sawyer/policy/trans_coffee_200_84.pth"
assert os.path.exists(ckpt_path)

In [ ]:
device = TorchUtils.get_torch_device(try_to_use_cuda=True)

# restore policy
policy, ckpt_dict = FileUtils.policy_from_checkpoint(ckpt_path=ckpt_path, device=device, verbose=True)
pass 

In [6]:
ckpt_dict = FileUtils.maybe_dict_from_checkpoint(ckpt_path=ckpt_path, ckpt_dict=None)
# algo name and config from model dict
algo_name, _ = FileUtils.algo_name_from_checkpoint(ckpt_dict=ckpt_dict)
config, _ = FileUtils.config_from_checkpoint(algo_name=algo_name, ckpt_dict=ckpt_dict, verbose=False)

config.unlock()
config.train.data ="/home/carl/data_sawyer/coffee/sawyer_coffee_gfs76.hdf5"
config.lock()

ObsUtils.initialize_obs_utils_with_config(config)
dataset_path = os.path.expanduser(config.train.data)
shape_meta = FileUtils.get_shape_metadata_from_dataset(
    dataset_path=config.train.data,
    all_obs_keys=config.all_obs_keys,
    verbose=True
)


============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['robot0_eef_pos']
using obs modality: rgb with keys: ['agentview_image', 'robot0_eye_in_hand_image']
using obs modality: depth with keys: []
using obs modality: scan with keys: []
obs key agentview_image with shape (84, 84, 3)
obs key robot0_eef_pos with shape (6,)
obs key robot0_eye_in_hand_image with shape (84, 84, 3)


In [7]:
def demo2actions(hdf5_file, demo_key, rollout_policy): 
    rollout_policy.start_episode()


    demo=hdf5_file['data'][demo_key]
    actions_real=demo['actions']
    obs=demo['obs']
    N=obs['robot0_eye_in_hand_image'].shape[0]

    obss=[]
    for i in range(N):
        ob={k:obs[k][i] for k in obs.keys()} 
        ob['robot0_eye_in_hand_image']=ob['robot0_eye_in_hand_image'].transpose(2,0,1)
        ob['agentview_image']=ob['agentview_image'].transpose(2,0,1)
        obss.append(ob)

    acs=[]
    for ob in obss: 
        ob = rollout_policy._prepare_observation(ob)
        ac = rollout_policy.policy.get_action(obs_dict=ob, goal_dict=None)
        ac=ac.detach().cpu().numpy()[0]
        acs.append(ac)

    actions_pred=np.array(acs)
    return actions_real, actions_pred

In [8]:
rclpy.init()

urdf_path = os.getcwd() +'/'+'sawyer.urdf'  #'/home/carl/sawyer_robot_ros2/src/teleop_script/sawyer.urdf'
robot = URDFModel(urdf_path)
jointMap = {name: ind for ind, name in enumerate(robot.jointNames)}


joint_names=['right_j0', 'right_j1', 'right_j2', 'right_j3', 'right_j4', 'right_j5', 'right_j6']
joint_inds=[jointMap[name] for name in joint_names]


node = Node('infer_live')
q = robot.getJoints()
ros_interface = ROSInterface(node, robot)
cmd_pub = node.create_publisher(JointCommand, '/robot/limb/right/joint_command', 10)
cmd_pub_en = node.create_publisher(Bool, '/script_ctrl', 10)

t1 = Thread(target=ros_interface.spin_thread)
t1.start()

sawyer
link: 0
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: base
link: 1
	cog: 0 0 0
	inertia: 1e-08 1e-08 1e-08     0     0     0
	mass: 0.0001
	name: torso
link: 2
	cog:    0    0 -0.5
	inertia:    5.06359    6.08689    4.96192 0.00105311   0.801996 0.00103417
	mass: 60.864
	name: pedestal
link: 3
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: controller_box
link: 4
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: pedestal_feet
link: 5
	cog:  -0.0006241 -2.8025e-05    0.065404
	inertia:   0.0067599   0.0067877   0.0074031  1.5888e-05 -6.1904e-07 -4.2024e-05
	mass: 2.0687
	name: right_arm_base_link
link: 6
	cog: 0.024366 0.010969  0.14363
	inertia:  0.053314  0.057902  0.023659 0.0080179  0.011734 0.0047093
	mass: 5.3213
	name: right_l0
link: 7
	cog:   0.0053207 -2.6549e-05      0.1021
	inertia:    0.011833   0.0082709   0.0049661  4.2124e-07  4.9425e-05 -4.4669e-06
	mass: 1.5795
	name: head
link: 8
	cog: 0 0 0
	inertia: 1e-08 1e-08 1e-08     0     0     0
	mass: 0.0001
	name

In [9]:
def set_script_ctrl(tof):
    msgB = Bool()
    msgB.data = tof
    cmd_pub_en.publish(msgB)

In [10]:
def get_ee(jointmsg):
    msg=jointmsg    
    q =  robot.getJoints() 
    for ind, name in enumerate(msg.name):
        if name in jointMap:
            q[jointMap[name]] = msg.position[ind]
    robot.setJoints(q)
    
    # T=robot.getBodyTransform('right_l6');
    T=robot.getBodyTransform('camera_link');
    xyz=T[0:3,3]
    e1 = rotm2euler.rotationMatrixToEulerAngles(T[0:3, 0:3])
    ee6=np.concatenate([xyz, e1])
    return ee6

def set_vels(vels):
    cmd_msg = JointCommand()
    cmd_msg.names = robot.jointNames
    cmd_msg.mode = cmd_msg.VELOCITY_MODE
    cmd_msg.header.stamp = rclpy.clock.Clock().now().to_msg()
    cmd_msg.velocity =vels 
    cmd_pub.publish(cmd_msg)

def goto_xyzrpy(target, delay=0.1, tol=0.05, max_step=100, gain=1.0, stop=True):
    ee6=get_ee(ros_interface.latest_joint_states)
    xd=ee6-target
    xd[3:6]=0 

    # print('xd: ', xd)

    alpha = .0001
    I = np.eye(q.shape[0])

    step=0
    while np.linalg.norm(xd)>tol:
        ee6=get_ee(ros_interface.latest_joint_states)
        xd=target-ee6 
        xd = xd * gain
        xd[3:6]=0        # only xyz, rpy not working yet

        J = robot.getJacobian('camera_link')
        
        qd = np.linalg.inv(J.T @ J + alpha * I) @ J.T @ xd

        set_vels(list(qd))
        time.sleep(delay)

        step = step+1 
        if step>=max_step:
            break 
    
    # stop
    if stop:
        set_vels([0.0]*10)

# apos=0
# def goto_q_pos_velocity_ctrl(target_q, alpha=0.7, tol=0.01,  delay=0.1, maxstep=50):
#     global apos
#     target_jointpos=target_q
#     cpos=robot.getJoints()
#     dpos=np.linalg.norm(target_jointpos-cpos)

#     step =0
#     while dpos>tol: 
#         cpos=robot.getJoints()
#         dpos=np.linalg.norm(target_jointpos-cpos)
          

#         qd =(target_jointpos-cpos)*alpha
#         apos=qd 
#         if np.max(qd) <0.001:
#             print('reached')
#             break 


#         set_vels(list(qd))
#         time.sleep(delay)
#         step +=1

#         if step>=maxstep:
#             break 
        
#     set_vels([0.0]*10)

apos=0
def goto_q_pos_velocity_ctrl(target_q, alpha=0.7, tol=0.01,  delay=0.1, maxstep=50):
    global apos
    target_jointpos=target_q
    cpos=robot.getJoints()
    dpos=np.linalg.norm(target_jointpos-cpos)

    set_script_ctrl(True)
    time.sleep(1)

    step =0
    while dpos>tol: 
        cpos=robot.getJoints()
        dpos=np.linalg.norm(target_jointpos-cpos)
          

        qd =(target_jointpos-cpos)*alpha
        apos=qd 
        if np.max(qd) <0.001:
            print('reached')
            break 


        set_vels(list(qd))
        time.sleep(delay)
        step +=1

        if step>=maxstep:
            break 
        
    set_vels([0.0]*10)

    set_script_ctrl(False)


### Robot safety check

In [12]:
pos1=robot.getJoints()
pos1

array([ 1.01325098,  0.10701465,  0.42728027, -1.49230664,  1.53894043,
       -1.16489453, -1.49287109,  2.73164355,  0.        ,  0.        ])

### move little by hand, then execute the following

In [12]:
goto_q_pos_velocity_ctrl(pos1, tol=0.005)

### init

In [26]:
init_coffee1 = np.array([ 1.01335352,  0.70604199,  0.42728027, -1.49230664,  1.53894043,
       -1.16489453, -1.49287109,  2.73164355,  0.04069973,  0.        ])

In [14]:
goto_q_pos_velocity_ctrl(init_coffee1, tol=0.005, maxstep=200)

### Trajectory playing test

In [13]:
dataset_path =  config.train.data 
print(f'dataset_path: {dataset_path}')

f = h5py.File(dataset_path, "r")
demos = list(f["data"].keys())
len(demos)

dataset_path: /home/carl/data_sawyer/coffee/sawyer_coffee_gfs76.hdf5


76

In [14]:
demo_name="demo_1"
demo=f['data'][demo_name]
actions=demo['actions']
ee_pos=demo['obs']['robot0_eef_pos']
ee_pos.shape

(679, 6)

In [15]:
f['data'][demo_name]['obs'].keys()

<KeysViewHDF5 ['agentview_image', 'robot0_eef_pos', 'robot0_eye_in_hand_image', 'robot0_joint_pos', 'robot0_joint_vel']>

In [16]:
goto_xyzrpy(ee_pos[0], tol=0.005) 

In [17]:
for i in range(len(ee_pos)):
    goto_xyzrpy(ee_pos[i], delay=0.001, tol=0.02) 

goto_xyzrpy(ee_pos[i], tol=0.005)  # last one with precision

In [18]:
goto_xyzrpy(ee_pos[-1], tol=0.005)

In [17]:
eepos_current=ee_pos[0]
eepos_current

array([ 0.41599406,  0.13853751,  0.12253274, -0.87225216,  1.5505728 ,
       -2.42463191])

### Robot is ready

In [18]:
policy.start_episode()

In [19]:
def get_current_obs():
    # img_wrist=cam_wrist.get_current_frame(scale=0.5)  
    # img_front=cam_front.get_current_frame(scale=0.5) 

    image_wrist=ros_interface.image_wrist
    image_front=ros_interface.image_front

    if image_front.sum()==0.0:
        print('image front None')
        return None
    if image_wrist.sum()==0:
        print('image wrist None')
        return None 

    scale=0.5
    img_wrist = cv2.resize(image_wrist, (int(image_wrist.shape[1]*scale), int(image_wrist.shape[0]*scale) ) , interpolation = cv2.INTER_AREA)
    img_front = cv2.resize(image_front, (int(image_front.shape[1]*scale), int(image_front.shape[0]*scale) ) , interpolation = cv2.INTER_AREA)


    # print('shape: ', image_wrist.shape, image_front.shape)
    msg=ros_interface.latest_joint_states
    gripper_isopen=ros_interface.latest_gripper_state

    ee=get_ee(msg)

    try:
        pos =np.array([msg.position[ind] for ind in joint_inds])
        vel =np.array([msg.velocity[ind] for ind in joint_inds])
        t=msg.header.stamp.sec+msg.header.stamp.nanosec*1e-9
    except Exception as e:
        # print('msg=', msg)
        # print('Error=',e, e.st)
        # traceback.print_exc()
        return None 

    img_wrist=cv2.cvtColor(img_wrist, cv2.COLOR_BGR2RGB)
    img_front=cv2.cvtColor(img_front, cv2.COLOR_BGR2RGB)

    img_wrist=cv2.resize(img_wrist, (84,84))
    img_front=cv2.resize(img_front, (84,84))


    ob={'agentview_image': img_front, 'robot0_eye_in_hand_image': img_wrist, 'robot0_joint_pos': pos, 'robot0_joint_vel':vel, 'robot0_eef_pos': ee}
    ob['robot0_eye_in_hand_image']=ob['robot0_eye_in_hand_image'].transpose(2,0,1)
    ob['agentview_image']=ob['agentview_image'].transpose(2,0,1)

    return ob

In [20]:
ob=get_current_obs()
ob!=None 

True

In [21]:
ob.keys()

dict_keys(['agentview_image', 'robot0_eye_in_hand_image', 'robot0_joint_pos', 'robot0_joint_vel', 'robot0_eef_pos'])

In [22]:
goto_xyzrpy(ee_pos[0], tol=0.005) 

In [23]:
eepos_start=ee_pos[0]
eepos_current=eepos_start
eepos_current

array([ 0.41599406,  0.13853751,  0.12253274, -0.87225216,  1.5505728 ,
       -2.42463191])

In [24]:
cv2.destroyAllWindows() 

In [27]:
goto_q_pos_velocity_ctrl(init_coffee1, tol=0.005, maxstep=200)

### try

In [28]:
frequency = 20.0   # similar to robomimic


eepos_start=ee_pos[0]
eepos_current=eepos_start


st=time.time()
acs=[]
for i in range(1000000):
    ob=get_current_obs()
    if ob==None:
        continue

    ob2 = policy._prepare_observation(ob)
    # ac = policy.policy.get_action(obs_dict=ob2, goal_dict=None)
    # ac=ac.detach().cpu().numpy()[0]
    ac=policy(ob2)
    print(f'i={i} ac={ac}')
    acs.append(ac)

    if ac[-1]>0.9:
        print('---------------------------------')
        print('grip now-----------------------')
        print('\n---------------------------------------')
        tb=Thread(target=beep_start)
        tb.start()
        # break 

    eepos_current[:3] += ac[:3]
    print(f'eepos_togo: {eepos_current}')

    goto_xyzrpy(eepos_current, tol=0.01, max_step=2, stop=False)
    


    image_front=ob['agentview_image'].transpose(2,1,0) 
    image_wrist=ob['robot0_eye_in_hand_image'].transpose(2,1,0)
    

    image=np.concatenate([image_wrist, image_front], axis=1) 
    image= cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    cv2.imshow('frame',image)
    if cv2.waitKey(1) == 27: 
        print('UI closed')
        break  # esc to quit

    time.sleep(1.0/frequency)

cv2.destroyAllWindows() 

/home/carl/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


i=0 ac=[-0.00250738  0.00328677 -0.00501301 -0.01303758  0.00836326 -0.01449115
  0.9997623 ]
---------------------------------
grip now-----------------------

---------------------------------------
eepos_togo: [ 0.41348668  0.14182428  0.11751973 -0.87225216  1.5505728  -2.42463191]
i=1 ac=[ 0.00164268 -0.00441422 -0.00154139  0.01002436  0.00549421  0.00598545
 -0.00046488]
eepos_togo: [ 0.41512936  0.13741006  0.11597833 -0.87225216  1.5505728  -2.42463191]
i=2 ac=[ 0.00155449 -0.00424421 -0.00182286  0.01014987  0.00543258  0.00594397
 -0.00023442]
eepos_togo: [ 0.41668384  0.13316585  0.11415548 -0.87225216  1.5505728  -2.42463191]
i=3 ac=[ 0.0013658  -0.00424736 -0.00181591  0.00983691  0.00557658  0.00565354
 -0.00031007]
eepos_togo: [ 0.41804965  0.12891849  0.11233956 -0.87225216  1.5505728  -2.42463191]
i=4 ac=[ 0.00135685 -0.00419346 -0.00166153  0.01009375  0.00541456  0.00592486
 -0.00033559]
eepos_togo: [ 0.4194065   0.12472504  0.11067804 -0.87225216  1.5505728  -2.424

KeyboardInterrupt: 

In [38]:
goto_xyzrpy(ee_pos[0], tol=0.005)  

In [29]:
ac

array([-3.6969720e-04,  1.2310240e-03,  3.0301284e-04, -1.6575159e-03,
        4.3890024e-05, -1.8973439e-03, -2.0936792e-04], dtype=float32)

In [42]:
cv2.destroyAllWindows() 

In [23]:
ob=get_current_obs()

In [24]:
ob2 = policy._prepare_observation(ob)

In [25]:
ob2['agentview_image'].shape

torch.Size([1, 3, 84, 84])

In [26]:
ac=policy(ob2)

/home/carl/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


In [27]:
ac

array([ 0.00154207, -0.00443313, -0.0015523 ,  0.01016796,  0.00538594,
        0.0060267 , -0.00034769], dtype=float32)

In [29]:
ac.shape

(7,)

In [1]:
ob2

NameError: name 'ob2' is not defined

In [ ]:
obs2={}
for key in obs.keys():
    obs2[key]=obs[key][0] 
    obs2[key]=np.expand_dims(obs2[key], axis=0)

In [ ]:
ac = policy.policy.get_action(obs_dict=ob2, goal_dict=None)

In [ ]:
ac=ac.detach().cpu().numpy()[0]
print(f'i={i} ac={ac}')